In [7]:
#!pip install --upgrade pip
#!pip install transformers==3.1.0

In [8]:
from transformers import pipeline
from os import listdir
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import f1_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import numpy as np

In [9]:
#classifier = pipeline("zero-shot-classification")
classifier = pipeline("zero-shot-classification", device=1) # to utilize GPU

Some weights of the model checkpoint at facebook/bart-large-mnli were not used when initializing BartForSequenceClassification: ['model.encoder.version', 'model.decoder.version']
- This IS expected if you are initializing BartForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BartForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


We can use this pipeline by passing in a sequence and a list of candidate labels. The pipeline assumes by default that only one of the candidate labels is true, returning a list of scores for each label which add up to 1.

In [10]:
path_screenplays_scenes='coref/csi-corpus/screenplay_summarization/scene_level_n_aspects'
eps=[]


for ep in listdir(path_screenplays_scenes):
    annotated_scenes=pd.read_csv(path_screenplays_scenes+'/'+ep)
    eps.append(ep.split('.csv')[0])
    
print(eps)    
method_used='tfidf'



    



['s05e10', 's03e03', 's01e19', 's01e23', 's02e15', 's02e01', 's05e22', 's02e09', 's05e21', 's04e23', 's04e09', 's05e12', 's03e12', 's05e03', 's04e22', 's03e21', 's02e04', 's03e19', 's04e14', 's04e06', 's01e07', 's04e21', 's05e17', 's03e08', 's04e10', 's04e05', 's05e05', 's05e06', 's01e20', 's04e15', 's02e06', 's01e08', 's04e12', 's05e08', 's03e05', 's02e10', 's05e13', 's03e11', 's01e13']


In [11]:


def get_results(df, classifier, topic_labels):
    df['topic'] = None
    df['score'] = None
    
    for i in tqdm(range(0, df.shape[0])):
        text = df.loc[i, 'scene_text']
        if text is not None:
            result = classifier(text, topic_labels)
            #df.loc[i, 'topic'] = str(str(result['labels'][0])+"+"+str(result['labels'][1]))
            #df.loc[i, 'score'] = str(str(result['scores'][0])+"+"+str(result['scores'][1]))
            df.loc[i, 'topic'] = result['labels'][0]
            df.loc[i, 'score_hf'] = result['scores'][0]
            #if result['labels'][0]=='crime scene':
                #df.loc[i, 'score_hf'] = result['scores'][0]
            #else:
                #df.loc[i, 'score_hf'] = -1
            
    return df


    
    

    




In [12]:
#eps=['s05e10']
for ep in eps:
    print(ep)
    df=pd.read_csv(ep+'_'+method_used+'_similarity.csv')
    print(df)
    
    

s05e10
    Unnamed: 0  scene_id                                         scene_text  \
0            0         0  [' ( Brass and Sofia stand around a dead body ...   
1            1         1  [' ( Sara walks up to David Phillips and Greg ...   
2            2         2  [' ( The door buzzes and opens for Warrick and...   
3            3         3  [' ( The prisoners are lined up in a row . )',...   
4            4         4  [" ( Catherine is standing next to the victim ...   
5            5         5  [' ( The lid is removed from the rubber bin Sa...   
6            6         6  [' ( Robbins goes over his findings with Sara ...   
7            7         7  [' ( Warrick and Det Vartann talk with Lt . )'...   
8            8         8  [' ( Sara is going through stacks of files . )...   
9            9         9  [' ( Warrick pushes the cart full of bagged ev...   
10          10        10  [' ( Sara and Robbins are scanning x-rays into...   
11          11        11  [' ( Sara and Brass

    Unnamed: 0  scene_id                                         scene_text  \
0            0         0  [' ( Martha Krell rolls her bag to the front d...   
1            1         1  [' ( The stairwell door opens and a man in a d...   
2            2         2  [' ( Officer car lights flash . )', ' ( Close-...   
3            3         3  [' ( Greg walks into the bathroom and finds Gr...   
4            4         4  [' ( Close-up of : Martha Krell is on the gurn...   
5            5         5  [' ( Brass interviews the Front Desk Clerk . )...   
6            6         6  [' ( Officers put up crime scene tape around t...   
7            7         7  [' ( David Phillips removes the thermometer an...   
8            8         8  [" ( Catherine takes a swab of Ty Hawkins ' be...   
9            9         9  [" ( Robbins goes over Martha Krell 's body wi...   
10          10        10  [' ( Open on the writing on the side of a file...   
11          11        11  [' ( Nick fingerprints Mar

In [13]:

f1_scores=[]
#eps=['s05e10']
for ep in eps:
    print(ep)
    df=pd.read_csv(ep+'_'+method_used+'_similarity.csv')
    sequence=df.scene_text
    #candidate_labels = ['Crime scene', 'Victim', 'Death cause', 'Perpetrator', 'Evidence', 'Motive']
    candidate_labels=['crime scene', 'victim', 'death cause', 'perpetrator', 'evidence', 'motive']
    #candidate_labels=['kill']
    #action, drama, horror and romance.
    #candidate_labels=['action', 'crime','romance','comedy']
    
    #candidate_labels=['Action','Animation','Biography','Comedy','Crime','Drama','Family','Fantasy','Horror','Mystery','Romance','SciFi','Thriller']
    #candidate_labels=['Action','Biography','Comedy','Crime','Drama','Family','Fantasy','Horror','Romance','SciFi','Thriller']
    #candidate_labels=['anger', 'sadness','joy',feat]
    first_results=get_results(df,classifier,candidate_labels)
    first_results[['prediction_score','score_hf']]=MinMaxScaler().fit_transform(first_results[['prediction_score','score_hf']])
                  
    #print(first_results)
    first_results['sum_score']=0.5*first_results['score_hf']+0.5*first_results['prediction_score']
    #print(first_results['sum_score'])
    #ranked=first_results[first_results.topic=='murder'].sort_values(by=['score'],ascending=False)
    first_results['prediction_class']=0
    #ranked['prediction_class']=0
    #ranked['prediction_class'].iloc[0:len(first_results[first_results.in_summary==1])]=1
    #predicted_pos= ranked['scene_id'][ranked['prediction_class']==1]
    #predicted_pos=list(predicted_pos)
    #print(predicted_pos)
    #first_results=first_results.sort_values(by=['sum_score'],ascending=False)
    first_results=first_results.sort_values(by=['score_hf'],ascending=False)
    first_results['prediction_class'].iloc[0:len(first_results[first_results.in_summary==1])]=1
    #first_results.to_csv('combined_scores_kill'+ep+'.csv',index=None)
    #first_results.loc[first_results['scene_id'].isin(predicted_pos), 'prediction_huggingface'] = 1 
    #print(first_results)
    f1_scores.append(f1_score(first_results.in_summary, first_results.prediction_class))
    
    
    

    
    
    

  0%|          | 0/34 [00:00<?, ?it/s]

s05e10


100%|██████████| 34/34 [00:12<00:00,  2.67it/s]
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
  0%|          | 0/39 [00:00<?, ?it/s]

s03e03


100%|██████████| 39/39 [00:15<00:00,  2.53it/s]
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
  0%|          | 0/36 [00:00<?, ?it/s]

...

100%|██████████| 34/34 [00:12<00:00,  2.66it/s]
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
  2%|▏         | 1/43 [00:00<00:04,  9.70it/s]

s03e11


100%|██████████| 43/43 [00:15<00:00,  2.86it/s]
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
  0%|          | 0/34 [00:00<?, ?it/s]

s01e13


100%|██████████| 34/34 [00:14<00:00,  2.34it/s]
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [332]:
#print(first_results[first_results.in_summary==1])


print(first_results[['aspects','in_summary','topic','score','prediction_class','sum_score','prediction','prediction_score']])



                 aspects  in_summary   topic score  prediction_class  \
10              Evidence           1  murder  None                 1   
21                  None           0  murder  None                 1   
23                  None           0  murder  None                 1   
6                   None           0  murder  None                 1   
1            Crime scene           1  murder  None                 1   
11                  None           0  murder  None                 1   
29                Motive           1  murder  None                 1   
12           Death cause           1  murder  None                 1   
19  Crime scene,Evidence           1  murder  None                 1   
24           Perpetrator           1  murder  None                 1   
27                  None           1  murder  None                 1   
2                   None           0  murder  None                 1   
8            Crime scene           1  murder  None              

In [14]:
print(np.mean(f1_scores))

print(len(f1_scores))



0.2292831477220617
39


In [34]:
f1_scores_bis=[]


for ep in eps:
    df=pd.read_csv('combined_scores_kill'+ep+'.csv')
    #df['max']=df[["prediction_score", "score_hf"]].max(axis=1)
    df['max']=0.5*df['score_hf']+0.5*df['prediction_score']
    df['max'][df.topic=='joy']=0
    df=df.sort_values(by=['max'],ascending=False)
    df['prediction_class']=0
    df['prediction_class'].iloc[0:len(df[df.in_summary==1])]=1
    #print(df[['in_summary','topic','score_hf','prediction_class','sum_score','prediction','prediction_score','max']])
    
            
            
    #first_results.to_csv('combined_scores'+ep+'.csv')
    #first_results.loc[first_results['scene_id'].isin(predicted_pos), 'prediction_huggingface'] = 1 
    #print(first_results)
    df=df.sort_values(by=['score_hf'],ascending=False)
    df['prediction_class'].iloc[0:len(df[df.in_summary==1])]=1
    #first_results.loc[first_results['scene_id'].isin(predicted_pos), 'prediction_huggingface'] = 1 
    #print(first_results)
    f1_scores_bis.append(f1_score(df.in_summary, df.prediction_class))
    


print(np.mean(f1_scores_bis))

#print(len(f1_scores))




/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: S

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:19

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:19

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: S

0.4877400347879802


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:19

In [279]:

f1_scores_emotions=[]
for ep in eps:
    df=pd.read_csv(ep+'_'+method_used+'_similarity.csv')
    sequence=df.scene_text
    #candidate_labels = ['Crime scene', 'Victim', 'Death cause', 'Perpetrator', 'Evidence', 'Motive']
    candidate_labels=['anger', 'anticipation', 'joy', 'trust', 'fear', 'surprise', 'sadness', 'disgust']
    first_results=get_results(df,classifier,candidate_labels)   
    first_results['prediction_class']=0
    first_results[first_results.topic=='Crime'].sort_values(by=['score'],ascending=False)[]

    ranked['prediction_class'].iloc[0:len(first_results[first_results.in_summary==1])]=1
    ranked.to_csv('murder_hum_class'+ep+'.csv')
    print(ranked)
    f1_scores_emotions.append(f1_score(ranked.in_summary, ranked.prediction_class))
   
classifier(sequence, candidate_labels, multi_class=True)


SyntaxError: invalid syntax (<ipython-input-279-c61821b32eb2>, line 9)

In [114]:
import numpy as np

sorted_f1 = f1_scores.copy()
sorted_f1.sort()
print(f1_scores[10])
print(eps[10])
ep=eps[10]
print(sorted_f1)
df=pd.read_csv('murder_hum_class'+ep+'.csv')

#print(df.scene_text[df.in_summary==0].iloc[4])
print(df)


1.0
s04e09
[0.19999999999999998, 0.2857142857142857, 0.28571428571428575, 0.3157894736842105, 0.3333333333333333, 0.36363636363636365, 0.4, 0.4, 0.4285714285714285, 0.43478260869565216, 0.45454545454545453, 0.47058823529411764, 0.5, 0.5, 0.5, 0.5, 0.5000000000000001, 0.5714285714285714, 0.6, 0.6, 0.6, 0.6153846153846153, 0.6153846153846153, 0.6153846153846153, 0.631578947368421, 0.6666666666666666, 0.6666666666666666, 0.6666666666666666, 0.6666666666666666, 0.7000000000000001, 0.7142857142857143, 0.7142857142857143, 0.7272727272727273, 0.7272727272727273, 0.7272727272727273, 0.7499999999999999, 0.7499999999999999, 0.8571428571428571, 1.0]
   Unnamed: 0  Unnamed: 0.1  scene_id  \
0           8             8         8   
1          30            30        30   

                                          scene_text  in_summary      aspects  \
0  [' ( Scene opens on a close up of two prints ,...           1  Death cause   
1  [' ( CATHERINE puts the bagged vice grips on t...           1   

To do multi-class classification, simply pass `multi_class=True`. In this case, the scores will be independent, but each will fall between 0 and 1.

In [ ]:
sequence = "Who are you voting for in 2020?"
candidate_labels = ["politics", "public health", "economics", "elections"]

classifier(sequence, candidate_labels, multi_class=True)

{'labels': ['politics', 'elections', 'public health', 'economics'],
 'scores': [0.972069501876831,
  0.967610776424408,
  0.03248710557818413,
  0.0061644683592021465],
 'sequence': 'Who are you voting for in 2020?'}

Here's an example of sentiment classification: 

In [ ]:
sequence = "I hated this movie. The acting sucked."
candidate_labels = ["positive", "negative"]

classifier(sequence, candidate_labels)

{'labels': ['negative', 'positive'],
 'scores': [0.9916268587112427, 0.00837317667901516],
 'sequence': 'I hated this movie. The acting sucked.'}

So how does this method work?

The underlying model is trained on the task of Natural Language Inference (NLI), which takes in two sequences and determines whether they contradict each other, entail each other, or neither.

This can be adapted to the task of zero-shot classification by treating the sequence which we want to classify as one NLI sequence (called the premise) and turning a candidate label into the other (the hypothesis). If the model predicts that the constructed premise _entails_ the hypothesis, then we can take that as a prediction that the label applies to the text. Check out [this blog post](https://joeddav.github.io/blog/2020/05/29/ZSL.html) for a more detailed explanation.

By default, the pipeline turns labels into hypotheses with the template `This example is {class_name}.`. This works well in many settings, but you can also customize this for your specific setting. Let's add another review to our above sentiment classification example that's a bit more challenging:

In [ ]:
sequences = [
    "I hated this movie. The acting sucked.",
    "This movie didn't quite live up to my high expectations, but overall I still really enjoyed it."
]
candidate_labels = ["positive", "negative"]

classifier(sequences, candidate_labels)

[{'labels': ['negative', 'positive'],
  'scores': [0.9916267991065979, 0.008373182266950607],
  'sequence': 'I hated this movie. The acting sucked.'},
 {'labels': ['negative', 'positive'],
  'scores': [0.8148515820503235, 0.1851484179496765],
  'sequence': "This movie didn't quite live up to my high expectations, but overall I still really enjoyed it."}]

The second example is a bit harder. Let's see if we can improve the results by using a hypothesis template which is more specific to the setting of review sentiment analysis. Instead of the default, `This example is {}.`, we'll use, `The sentiment of this review is {}.` (where `{}` is replaced with the candidate class name)

In [ ]:
sequences = [
    "I hated this movie. The acting sucked.",
    "This movie didn't quite live up to my high expectations, but overall I still really enjoyed it."
]
candidate_labels = ["positive", "negative"]
hypothesis_template = "The sentiment of this review is {}."

classifier(sequences, candidate_labels, hypothesis_template=hypothesis_template)

[{'labels': ['negative', 'positive'],
  'scores': [0.9890093207359314, 0.010990672744810581],
  'sequence': 'I hated this movie. The acting sucked.'},
 {'labels': ['positive', 'negative'],
  'scores': [0.9581228494644165, 0.0418771356344223],
  'sequence': "This movie didn't quite live up to my high expectations, but overall I still really enjoyed it."}]

By providing a more precise hypothesis template, we are able to see a more accurate classification of the second review.

> Note that sentiment classification is used here just as an illustrative example. The [Hugging Face Model Hub](https://huggingface.co/models?filter=text-classification) has a number of models trained specifically on sentiment tasks which can be used instead.

#### Update: Zero-shot classification in 100 languages

Interested in using the pipeline for languages other than English? We've trained a cross-lingual model on top of XLM RoBERTa which you can use by passing `model='joeddav/xlm-roberta-large-xnli'` when creating the pipeline:

In [ ]:
classifier = pipeline("zero-shot-classification", model='joeddav/xlm-roberta-large-xnli')

You can use it with any combination of languages. For example, let's classify a Russian sentence with English candidate labels:

In [ ]:
sequence = "За кого вы голосуете в 2020 году?" # translation: "Who are you voting for in 2020?"
candidate_labels = ["Europe", "public health", "politics"]

classifier(sequence, candidate_labels)

{'labels': ['politics', 'Europe', 'public health'],
 'scores': [0.9048484563827515, 0.05722189322113991, 0.03792969882488251],
 'sequence': 'За кого вы голосуете в 2020 году?'}

Now let's do the same but with the labels in French:



In [ ]:
sequence = "За кого вы голосуете в 2020 году?" # translation: "Who are you voting for in 2020?"
candidate_labels = ["Europe", "santé publique", "politique"]

classifier(sequence, candidate_labels)

{'labels': ['politique', 'Europe', 'santé publique'],
 'scores': [0.9726154804229736, 0.017128489911556244, 0.010256024077534676],
 'sequence': 'За кого вы голосуете в 2020 году?'}

As we discussed in the last section, the default hypothesis template is the English, `This text is {}.`. If you are working strictly within one language, it may be worthwhile to translate this to the language you are working with:

In [ ]:
sequence = "¿A quién vas a votar en 2020?"
candidate_labels = ["Europa", "salud pública", "política"]
hypothesis_template = "Este ejemplo es {}."

classifier(sequence, candidate_labels, hypothesis_template=hypothesis_template)

{'labels': ['política', 'Europa', 'salud pública'],
 'scores': [0.9109585881233215, 0.05954807624220848, 0.029493311420083046],
 'sequence': '¿A quién vas a votar en 2020?'}

The model is fine-tuned on XNLI which includes 15 languages: Arabic, Bulgarian, Chinese, English, French, German, Greek, Hindi, Russian, Spanish, Swahili, Thai, Turkish, Urdu, and Vietnamese. The base model is trained on 85 more, so the model will work to some degree for any of those in the XLM RoBERTa training corpus (see the full list in appendix A of the [XLM Roberata paper](https://arxiv.org/abs/1911.02116)).

See the [model page](https://huggingface.co/joeddav/xlm-roberta-large-xnli) for more.